In [1]:
# import the necessary packages
import skimage
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from skimage import feature
from imutils import build_montages
from imutils import paths
import numpy as np
import cv2
import os
import pickle #importing the pickle file

# New Section

In [2]:
def load_split(path):
    #grab the list of images in the input directory , then initialize
    # the list of data ( i.e. , images ) and class labels
    imagePaths = list(paths.list_images(path))
    data = [ ]
    labels = [ ]
   #loop over the image paths
    for imagePath in imagePaths :
        #extract the class label from the filename
        label = imagePath.split(os.path.sep)[-2]
       #load the input image , convert it to grayscale , and resize
       #it to 200x200 pixels , ignoring aspect ratio
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image , (200 , 200))
        #threshold the image such that the drawing appears as white
        #on a black background
        image = cv2.threshold(image , 0 , 255 , cv2.THRESH_BINARY_INV | cv2 . THRESH_OTSU)[ 1 ]
        # quantify the image
        features = quantify_image(image)
        # update the data and labels lists , respectively
        data.append(features)
        labels.append(label)
   # return the data and labels
    return (np.array(data) , np.array(labels))

In [4]:
# define the path to the training and testing directories
trainingPath = r"dataset\spiral\training"
testingPath = r"dataset\spiral\testing"
# loading the training and testing data
print ( " [ INFO ] loading data ... " )
(X_train , y_train) = load_split(trainingPath)
(X_test , y_test) = load_split(testingPath)



 [ INFO ] loading data ... 


In [3]:
def quantify_image(image):
    # compute the histogram 
    features = feature.hog(image, orientations=9,
    pixels_per_cell=(10, 10), cells_per_block=(2, 2),
    transform_sqrt=True, block_norm="L1")

    # return the feature vector
    return features


In [5]:
# encode the labels as integers
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
print(X_train.shape,y_train.shape)

(72, 12996) (72,)


In [6]:
print( " [ INFO ] training model " )
model = RandomForestClassifier(n_estimators = 100 )
model.fit(X_train , y_train)

 [ INFO ] training model 


RandomForestClassifier()

In [7]:
testPaths = list(paths.list_images(testingPath))
idxs = np.arange(0 , len(testPaths))
idxs = np.random.choice(idxs , size=(25,), replace=False)
images = []

In [8]:
for i in idxs:
    image = cv2.imread(testPaths[i])
    output = image.copy()
    output = cv2.resize(output , (128,128))
    image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image , (200,200))
    image = cv2.threshold(image , 0 , 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    

In [9]:
features = quantify_image(image)
preds = model.predict([features])
label = le.inverse_transform(preds)[0]

color = (0 , 255 , 0) if label == "healthy" else (0 , 0 ,255)

cv2.putText(output , label , (3, 20) , cv2.FONT_HERSHEY_SIMPLEX , 0.5,color , 2)
images.append(output)




In [ ]:
montage = build_montages(images , (128,128) ,(5,5))[0]
cv2.imshow("OUTPUT",montage)
cv2.waitKey(0)

In [10]:
predictions = model.predict(X_test)

cm = confusion_matrix(y_test , predictions).flatten()
print(cm)

(tn , fp , fn , tp) = cm

accuracy  = (tp + tn) / float(cm.sum())

print(accuracy)



[13  2  5 10]
0.7666666666666667


In [11]:
pickle.dump(model , open('parkinsons.pkl','wb'))

In [9]:
# import numpy as np
# import cv2
# import os
# import argparse
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import confusion_matrix
# from skimage import feature
# from imutils import build_montages
# from imutils import paths



# def load_split(path):
#     imagePaths = list(paths.list_images(path))
#     data = []
#     labels = []

#     for imagePath in imagePaths:
#     # extract the class label from the filename
#     label = imagePath.split(os.path.sep)[-2]

#     image = cv2.imread(imagePath)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     image = cv2.resize(image, (200, 200))

#     image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

#     features = quantify_image(image)

#     # update the data and labels lists
#     data.append(features)
#     labels.append(label)

#     return (np.array(data), np.array(labels))

# # argument parser 
# ap = argparse.ArgumentParser()
# ap.add_argument("-d", "--dataset", required=True,help="path to input dataset")
# ap.add_argument("-t", "--trials", type=int, default=5,help="# of trials to run")
# args = vars(ap.parse_args())

# trainingPath = os.path.sep.join([args["dataset"], "training"])
# testingPath = os.path.sep.join([args["dataset"], "testing"])

# print("loading data...")
# (trainX, trainY) = load_split(trainingPath)
# (testX, testY) = load_split(testingPath)

# le = LabelEncoder()
# trainY = le.fit_transform(trainY)
# testY = le.transform(testY)

# trials = {}

# for i in range(0, args["trials"]):
#     # training
#     print("training model {} of {}...".format(i + 1,
#     args["trials"]))
#     model = RandomForestClassifier(n_estimators=100)
#     model.fit(trainX, trainY)

#     predictions = model.predict(testX)
#     metrics = {}

#     cm = confusion_matrix(testY, predictions).flatten()
#     (tn, fp, fn, tp) = cm
#     metrics["acc"] = (tp + tn) / float(cm.sum())
#     metrics["sensitivity"] = tp / float(tp + fn)
#     metrics["specificity"] = tn / float(tn + fp)

# 	for (k, v) in metrics.items():
# 		l = trials.get(k, [])
# 		l.append(v)
# 		trials[k] = l

# for metric in ("acc", "sensitivity", "specificity"):
# 	values = trials[metric]
# 	mean = np.mean(values)
# 	std = np.std(values)

# 	# computed metrics for the statistic
# 	print(metric)
# 	print("=" * len(metric))
# 	print("u={:.4f}, o={:.4f}".format(mean, std))
# 	print("")

# testingPaths = list(paths.list_images(testingPath))
# idxs = np.arange(0, len(testingPaths))
# idxs = np.random.choice(idxs, size=(25,), replace=False)
# images = []

# # loop over the testing samples
# for i in idxs:
# 	image = cv2.imread(testingPaths[i])
# 	output = image.copy()
# 	output = cv2.resize(output, (128, 128))

# 	image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# 	image = cv2.resize(image, (200, 200))
# 	image = cv2.threshold(image, 0, 255,
# 		cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

# 	features = quantify_image(image)
# 	preds = model.predict([features])
# 	label = le.inverse_transform(preds)[0]

# 	color = (0, 255, 0) if label == "healthy" else (0, 0, 255)
# 	cv2.putText(output, label, (3, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
# 		color, 2)
# 	images.append(output)

# montage = build_montages(images, (128, 128), (5, 5))[0]

# cv2.imshow("Output", montage)
# cv2.waitKey(0)

usage: ipykernel_launcher.py [-h] -d DATASET [-t TRIALS]
ipykernel_launcher.py: error: the following arguments are required: -d/--dataset


SystemExit: 2

C:\Users\aasim\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
